# Example: Training a dense MNIST model


In [1]:
#Import requisite packages
#data manipulation
import numpy as np
import tempfile

#pytorch for training neural network
import torch, torch.onnx
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

#pyomo for optimization
import pyomo.environ as pyo

#omlt for interfacing our neural network with pyomo
from omlt import OmltBlock
from omlt.neuralnet import FullSpaceNNFormulation
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds

## Import the Data and Train a Neural Network

We begin by loading the MNIST dataset as `DataLoader` objects with pre-set training and testing batch sizes:


In [2]:
#set training and test batch sizes
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}

#build DataLoaders for training and test sets
dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor())
dataset2 = datasets.MNIST('../data', train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Next, we define the structure of the dense neural network model:

In [3]:
hidden_size = 50

class Net(nn.Module):
    #define layers of neural network
    def __init__(self):
        super().__init__()
        self.hidden1  = nn.Linear(784, hidden_size)
        self.hidden2  = nn.Linear(hidden_size, hidden_size)
        self.output  = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    #define forward pass of neural network
    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)      
        return x

We next define simple functions for training and testing the neural network:

In [4]:
#training function computes loss and its gradient on batch, and prints status after every 200 batches
def train(model, train_loader, optimizer, epoch):
    model.train(); criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.view(-1, 28*28))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

#testing function computes loss and prints overall model accuracy on test set
def test(model, test_loader):
    model.eval(); criterion = nn.NLLLoss( reduction='sum')
    test_loss = 0; correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.view(-1, 28*28))
            test_loss += criterion(output, target).item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))            

Finally, we train the neural network on the dataset.
Training here is performed using the `Adadelta` optimizer for five epochs.

In [5]:
#define model and optimizer
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

#train neural network for five epochs
for epoch in range(5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.333585
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.212676
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.666304
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.204922
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.330610

Test set: Average loss: 0.2147, Accuracy: 9334/10000 (93%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.164354
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.295505
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.157278
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.055990
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.080962

Test set: Average loss: 0.1254, Accuracy: 9611/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.079152
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.111729
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.138119
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.025251
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.022882

Test set: Average loss: 0.1086, Accuracy: 9685/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.286376
Train Epoch: 3 [12800/60000 (21%)]	Lo

In [6]:
# save the pytorch model 

torch.save(model.state_dict(), 'classification_model.pth')